<a href="https://colab.research.google.com/github/DumontHenry/exercise_PySpark/blob/main/PySpark_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.4.1
!pip install findspark==2.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285388 sha256=472a2cca39d5c3c8ff45c0c5db9715290f95da170d6ad1fd7f412a1cebc4bcc4
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
import pyspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PySpark 101 Exercises").getOrCreate()
print(spark.version)


3.4.1


In [ ]:
# withColumn = By using PySpark withColumn() on a DataFrame, we can cast or change the data type of a column.
# withColumn =  PySpark withColumn() function of DataFrame can also be used to change the value of an existing column.$
# withColumn = To add/create a new column, specify the first argument with a name you want your new column to be and use the second argument to assign a value by applying an operation on an existing column
# viwthColumn = In order to create a new column, pass the column name you wanted to the first argument of withColumn() transformation function.


# withColumnRenamed() = Though you cannot rename a column using withColumn, still I wanted to cover this as renaming is one of the common operations we perform on DataFrame. To rename an existing column use withColumnRenamed() function on DataFrame.
#  SparkSession.builder.appName('SparkByExamples.com').getOrCreate() = In PySpark, SparkSession.builder.appName() sets a name for your Spark application. This name is useful for identifying your application in Spark's web UI or logs

In [ ]:
df = spark.createDataFrame([
("Alice", 1),
("Bob", 2),
("Charlie", 3),
], ["Name", "Value"])

df.show()

+-------+-----+
|   Name|Value|
+-------+-----+
|  Alice|    1|
|    Bob|    2|
|Charlie|    3|
+-------+-----+



In [3]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number, monotonically_increasing_id

# Applying orderBy() and monotonically_increasing_id()
window_spec = Window.orderBy(monotonically_increasing_id())

# Add a new column "row_number" using row_number() over the specified window
result_df = df.withColumn("Index", row_number().over(window_spec)-1)

# Show the result
result_df.show()

NameError: name 'df' is not defined

In [ ]:
list1 = ["a", "b", "c", "d"]
list2 = [1, 2, 3, 4]

In [ ]:
# Create an RDD from the lists and convert it to a DataFrame
rdd = spark.sparkContext.parallelize(list(zip(list1, list2)))
df = rdd.toDF(["Column1", "Column2"])

# Show the DataFrame
df.show()

+-------+-------+
|Column1|Column2|
+-------+-------+
|      a|      1|
|      b|      2|
|      c|      3|
|      d|      4|
+-------+-------+



In [ ]:
list_A = [1, 2, 3, 4, 5]
list_B = [4, 5, 6, 7, 8]

In [ ]:
sc = spark.sparkContext

rdd_A = sc.parallelize(list_A)
rdd_B = sc.parallelize(list_B)

result_rdd = rdd_A.subtract(rdd_B)
# Collect result
result_list = result_rdd.collect()
print(result_list)

[1, 2, 3]


In [ ]:
list_A = [1, 2, 3, 4, 5]
list_B = [4, 5, 6, 7, 8]

In [ ]:
sc = spark.sparkContext

rdd_A = sc.parallelize(list_A)
rdd_B = sc.parallelize(list_B)

result_rdd_A = rdd_A.subtract(rdd_B)
result_rdd_B = rdd_B.subtract(rdd_A)

result_rdd = result_rdd_A.union(result_rdd_B)
# Collect result
result_list = result_rdd.collect()
print(result_list)

[1, 2, 3, 8, 6, 7]


In [ ]:
data = [("A", 10), ("B", 20), ("C", 30), ("D", 40), ("E", 50), ("F", 15), ("G", 28), ("H", 54), ("I", 41), ("J", 86)]
df = spark.createDataFrame(data, ["Name", "Age"])

df.show()

+----+---+
|Name|Age|
+----+---+
|   A| 10|
|   B| 20|
|   C| 30|
|   D| 40|
|   E| 50|
|   F| 15|
|   G| 28|
|   H| 54|
|   I| 41|
|   J| 86|
+----+---+



In [ ]:
quantiles = df.approxQuantile("Age", [0.0, 0.25, 0.5, 0.75, 1.0], 0.01)
print(quantiles)
print("Min: ", quantiles[0])
print("25th percentile: ", quantiles[1])
print("Median: ", quantiles[2])
print("75th percentile: ", quantiles[3])
print("Max: ", quantiles[4])

[10.0, 20.0, 30.0, 50.0, 86.0]
Min:  10.0
25th percentile:  20.0
Median:  30.0
75th percentile:  50.0
Max:  86.0


In [ ]:
from pyspark.sql import Row

# Sample data
data = [
Row(name='John', job='Engineer'),
Row(name='John', job='Engineer'),
Row(name='Mary', job='Scientist'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Scientist'),
Row(name='Sam', job='Doctor'),
]

# create DataFrame
df = spark.createDataFrame(data)

# show DataFrame
df.show()

In [ ]:
df.select('name', 'job').distinct().collect()
####
df.groupby('job').count().show()
df.groupby('name').count().show()

+---------+-----+
|      job|count|
+---------+-----+
|Scientist|    2|
| Engineer|    4|
|   Doctor|    1|
+---------+-----+

+----+-----+
|name|count|
+----+-----+
|Mary|    1|
|John|    2|
| Bob|    3|
| Sam|    1|
+----+-----+



In [ ]:
from pyspark.sql import Row

# Sample data
data = [
Row(name='John', job='Engineer'),
Row(name='John', job='Engineer'),
Row(name='Mary', job='Scientist'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Engineer'),
Row(name='Bob', job='Scientist'),
Row(name='Sam', job='Doctor'),
]

# create DataFrame
df = spark.createDataFrame(data)

# show DataFrame
df.show()

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|   Doctor|
+----+---------+



In [ ]:
from pyspark.sql.functions import col, when

# Get the top 2 most frequent jobs
top_2_jobs= df.groupby('job').count().orderBy('count', ascending=False).limit(2).select('job').rdd.flatMap(lambda x: x).collect()
# Replace all but the top 2 most frequent jobs with 'Other'
df = df.withColumn('job', when(col('job').isin(top_2_jobs), col('job')).otherwise('Other'))
df.show()

+----+---------+
|name|      job|
+----+---------+
|John| Engineer|
|John| Engineer|
|Mary|Scientist|
| Bob| Engineer|
| Bob| Engineer|
| Bob|Scientist|
| Sam|    Other|
+----+---------+



In [ ]:
# Assuming df is your DataFrame
df = spark.createDataFrame([
("A", 1, None),
("B", None, "123" ),
("B", 3, "456"),
("D", None, None),
], ["Name", "Value", "id"])

df.show()

+----+-----+----+
|Name|Value|  id|
+----+-----+----+
|   A|    1|null|
|   B| null| 123|
|   B|    3| 456|
|   D| null|null|
+----+-----+----+



In [ ]:
df_1 = df.dropna(subset=['Value'], how='all')
df_1.show()

+----+-----+----+
|Name|Value|  id|
+----+-----+----+
|   A|    1|null|
|   B|    3| 456|
+----+-----+----+



In [ ]:
# suppose you have the following DataFrame
df = spark.createDataFrame([(1, 2, 3), (4, 5, 6)], ["col1", "col2", "col3"])

# old column names
old_names = ["col1", "col2", "col3"]

# new column names
new_names = ["new_col1", "new_col2", "new_col3"]

df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   1|   2|   3|
|   4|   5|   6|
+----+----+----+



In [ ]:
for old_name, new_name in zip(old_names, new_names):
  df = df.withColumnRenamed(old_name, new_name )
df.show()

+--------+--------+--------+
|new_col1|new_col2|new_col3|
+--------+--------+--------+
|       1|       2|       3|
|       4|       5|       6|
+--------+--------+--------+



In [ ]:
from pyspark.sql.functions import rand
from pyspark.ml.feature import Bucketizer

# Create a DataFrame with a single column "values" filled with random numbers
num_items = 100
df = spark.range(num_items).select(rand(seed=42).alias("values"))

df.show(5)

+-------------------+
|             values|
+-------------------+
|  0.619189370225301|
| 0.5096018842446481|
| 0.8325259388871524|
|0.26322809041172357|
| 0.6702867696264135|
+-------------------+
only showing top 5 rows



In [ ]:
num_buckets = 10  # Number of buckets
quantiles = df.stat.approxQuantile("values",[i/num_buckets for i in range(num_buckets+1)], 0.01)

# Create the Bucketizer
bucketizer = Bucketizer(splits=quantiles, inputCol="values", outputCol="buckets")

# Apply the Bucketizer
df_buck = bucketizer.transform(df)

#Frequency table
df_buck.groupBy("buckets").count().show()

# Show the original and bucketed values
df_buck.show(5)


+-------+-----+
|buckets|count|
+-------+-----+
|    8.0|   10|
|    0.0|    8|
|    7.0|   10|
|    1.0|   10|
|    4.0|   10|
|    3.0|   10|
|    2.0|   10|
|    6.0|   10|
|    5.0|   10|
|    9.0|   12|
+-------+-----+

+-------------------+-------+
|             values|buckets|
+-------------------+-------+
|  0.619189370225301|    4.0|
| 0.5096018842446481|    4.0|
| 0.8325259388871524|    8.0|
|0.26322809041172357|    2.0|
| 0.6702867696264135|    5.0|
+-------------------+-------+
only showing top 5 rows



In [ ]:
# Example DataFrame
data = [("A", "X"), ("A", "Y"), ("A", "X"), ("B", "Y"), ("B", "X"), ("C", "X"), ("C", "X"), ("C", "Y")]
df = spark.createDataFrame(data, ["category1", "category2"])

df.show()


+---------+---------+
|category1|category2|
+---------+---------+
|        A|        X|
|        A|        Y|
|        A|        X|
|        B|        Y|
|        B|        X|
|        C|        X|
|        C|        X|
|        C|        Y|
+---------+---------+



In [ ]:
df.cube("category1").count().show()
"""
The cube function in PySpark is used to perform data aggregation by generating all possible combinations of grouping columns, similar to how the groupby function works.
However, cube goes a step further by also including aggregations for all subsets of the specified grouping columns, as well as the grand total (aggregation across all data).
This means that if you apply cube on multiple columns, it will generate aggregations for each individual column, all possible pairs, all possible triplets, and so on, along with the overall total.
For example, in the code you provided, df.cube("category1").count().show(), the cube function is applied to the "category1" column.
This will generate counts for each distinct value in "category1" as well as a total count for all categories combined.
"""

+---------+-----+
|category1|count|
+---------+-----+
|        B|    2|
|     null|    8|
|        A|    3|
|        C|    3|
+---------+-----+



In [ ]:
# Contingency table
df.crosstab('category1', 'category2').show()
"""
The crosstab function in PySpark is used to compute a contingency table (also known as a cross-tabulation) for two columns of a DataFrame.
 It essentially creates a frequency table that shows the distribution of values in one column, categorized by the values in another column.
In your code, df.crosstab('category1', 'category2').show(), it generates a table that shows how many times each combination of values from "category1" and "category2" appears in your DataFrame.
This allows you to see the relationship or association between those two categorical variables
"""

+-------------------+---+---+
|category1_category2|  X|  Y|
+-------------------+---+---+
|                  B|  1|  1|
|                  C|  2|  1|
|                  A|  2|  1|
+-------------------+---+---+



In [ ]:
from pyspark.sql.functions import rand

# Generate a DataFrame with a single column "id" with 10 rows
df = spark.range(10)
# Generate a random float between 0 and 1, scale and shift it to get a random integer between 1 and 10
df = df.withColumn("random", ((rand(seed=42) * 10) + 1).cast("int"))
# Show the DataFrame
df.show()

+---+------+
| id|random|
+---+------+
|  0|     7|
|  1|     6|
|  2|     9|
|  3|     3|
|  4|     7|
|  5|     9|
|  6|     7|
|  7|     3|
|  8|     3|
|  9|     7|
+---+------+



In [ ]:
from pyspark.sql.functions import col, when

df = df.withColumn("is_multiple_of_3", when(col("random")%3==0, 1).otherwise(0))
df.show()

+---+------+----------------+
| id|random|is_multiple_of_3|
+---+------+----------------+
|  0|     7|               0|
|  1|     6|               1|
|  2|     9|               1|
|  3|     3|               1|
|  4|     7|               0|
|  5|     9|               1|
|  6|     7|               0|
|  7|     3|               1|
|  8|     3|               1|
|  9|     7|               0|
+---+------+----------------+



In [ ]:
from pyspark.sql.functions import rand

# Generate a DataFrame with a single column "id" with 10 rows
df = spark.range(10)

# Generate a random float between 0 and 1, scale and shift it to get a random integer between 1 and 10
df = df.withColumn("random", ((rand(seed=42) * 10) + 1).cast("int"))

# Show the DataFrame
df.show()

pos = [0, 4, 8, 5]

+---+------+
| id|random|
+---+------+
|  0|     7|
|  1|     6|
|  2|     9|
|  3|     3|
|  4|     7|
|  5|     9|
|  6|     7|
|  7|     3|
|  8|     3|
|  9|     7|
+---+------+



In [ ]:
pos = [0, 4, 8, 5]

# Define window specification
w = Window.orderBy(monotonically_increasing_id())

# Add index
df = df.withColumn("index", row_number().over(w) - 1)

df.show()

# Filter the DataFrame based on the specified positions
df_filtered = df.filter(df.index.isin(pos))

df_filtered.show()

+---+------+-----+
| id|random|index|
+---+------+-----+
|  0|     7|    0|
|  1|     6|    1|
|  2|     9|    2|
|  3|     3|    3|
|  4|     7|    4|
|  5|     9|    5|
|  6|     7|    6|
|  7|     3|    7|
|  8|     3|    8|
|  9|     7|    9|
+---+------+-----+

+---+------+-----+
| id|random|index|
+---+------+-----+
|  0|     7|    0|
|  4|     7|    4|
|  5|     9|    5|
|  8|     3|    8|
+---+------+-----+



In [ ]:
df_A = spark.createDataFrame([("apple", 3, 5), ("banana", 1, 10), ("orange", 2, 8)], ["Name", "Col_1", "Col_2"])
df_A.show()

# Create DataFrame for region B
df_B = spark.createDataFrame([("apple", 3, 5), ("banana", 1, 15), ("grape", 4, 6)], ["Name", "Col_1", "Col_3"])
df_B.show()

+------+-----+-----+
|  Name|Col_1|Col_2|
+------+-----+-----+
| apple|    3|    5|
|banana|    1|   10|
|orange|    2|    8|
+------+-----+-----+

+------+-----+-----+
|  Name|Col_1|Col_3|
+------+-----+-----+
| apple|    3|    5|
|banana|    1|   15|
| grape|    4|    6|
+------+-----+-----+



In [ ]:
df= df_A.union(df_B)
df.show()

+------+-----+-----+
|  Name|Col_1|Col_2|
+------+-----+-----+
| apple|    3|    5|
|banana|    1|   10|
|orange|    2|    8|
| apple|    3|    5|
|banana|    1|   15|
| grape|    4|    6|
+------+-----+-----+



In [4]:
# Assume you have a DataFrame df with two columns "actual" and "predicted"
# For the sake of example, we'll create a sample DataFrame
data = [(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)]
df = spark.createDataFrame(data, ["actual", "predicted"])

df.show()

+------+---------+
|actual|predicted|
+------+---------+
|     1|        1|
|     2|        4|
|     3|        9|
|     4|       16|
|     5|       25|
+------+---------+



In [9]:
from pyspark.sql.functions import col, when
df= df.withColumn("squarred_error", pow(col("actual")-col("predicted"), 2))
mse = df.agg({"squarred_error": "avg"}).collect()[0][0]
df.show()
print(f"Mean Squared Error (MSE) = {mse}")

+------+---------+--------------+
|actual|predicted|squarred_error|
+------+---------+--------------+
|     1|        1|           0.0|
|     2|        4|           4.0|
|     3|        9|          36.0|
|     4|       16|         144.0|
|     5|       25|         400.0|
+------+---------+--------------+

Mean Squared Error (MSE) = 116.8


# **17. How to convert the first character of each element in a series to uppercase?**

In [10]:
# Suppose you have the following DataFrame
data = [("john",), ("alice",), ("bob",)]
df = spark.createDataFrame(data, ["name"])

df.show()

+-----+
| name|
+-----+
| john|
|alice|
|  bob|
+-----+



In [12]:
from pyspark.sql.functions import *
df.withColumn("name", initcap(col("name"))).show()

+-----+
| name|
+-----+
| John|
|Alice|
|  Bob|
+-----+



# **18. How to compute summary statistics for all columns in a dataframe**

In [13]:
# For the sake of example, we'll create a sample DataFrame
data = [('James', 34, 55000),
('Michael', 30, 70000),
('Robert', 37, 60000),
('Maria', 29, 80000),
('Jen', 32, 65000)]

df = spark.createDataFrame(data, ["name", "age" , "salary"])

df.show()

+-------+---+------+
|   name|age|salary|
+-------+---+------+
|  James| 34| 55000|
|Michael| 30| 70000|
| Robert| 37| 60000|
|  Maria| 29| 80000|
|    Jen| 32| 65000|
+-------+---+------+



In [14]:
summary = df.summary()
summary.show()

+-------+------+-----------------+-----------------+
|summary|  name|              age|           salary|
+-------+------+-----------------+-----------------+
|  count|     5|                5|                5|
|   mean|  null|             32.4|          66000.0|
| stddev|  null|3.209361307176242|9617.692030835671|
|    min| James|               29|            55000|
|    25%|  null|               30|            60000|
|    50%|  null|               32|            65000|
|    75%|  null|               34|            70000|
|    max|Robert|               37|            80000|
+-------+------+-----------------+-----------------+



# **19. How to calculate the number of characters in each word in a column?**

In [15]:
# Suppose you have the following DataFrame
data = [("john",), ("alice",), ("bob",)]
df = spark.createDataFrame(data, ["name"])

df.show()

+-----+
| name|
+-----+
| john|
|alice|
|  bob|
+-----+



In [17]:
import pyspark.sql.functions as F
df.select(F.length("name")).show()
df.withColumn("length", F.length("name")).show()

+------------+
|length(name)|
+------------+
|           4|
|           5|
|           3|
+------------+

+-----+------+
| name|length|
+-----+------+
| john|     4|
|alice|     5|
|  bob|     3|
+-----+------+



# **20 How to compute difference of differences between consecutive numbers of a column?**

In [18]:
# For the sake of example, we'll create a sample DataFrame
data = [('James', 34, 55000),
('Michael', 30, 70000),
('Robert', 37, 60000),
('Maria', 29, 80000),
('Jen', 32, 65000)]

df = spark.createDataFrame(data, ["name", "age" , "salary"])

df.show()

+-------+---+------+
|   name|age|salary|
+-------+---+------+
|  James| 34| 55000|
|Michael| 30| 70000|
| Robert| 37| 60000|
|  Maria| 29| 80000|
|    Jen| 32| 65000|
+-------+---+------+



In [23]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df = df.withColumn("id", F.monotonically_increasing_id())
window = Window.orderBy("id")
df = df.withColumn("data", F.lag(F.col("salary"), 1, 0).over(window))
df = df.withColumn("data", F.col("salary") - F.lag(F.col("salary"), 1, 0).over(window))
df= df.drop("id")
df.show()


+-------+---+------+------+----------+------+
|   name|age|salary|  data|prev_value|  diff|
+-------+---+------+------+----------+------+
|  James| 34| 55000| 55000|      null|     0|
|Michael| 30| 70000| 15000|     55000| 15000|
| Robert| 37| 60000|-10000|     70000|-10000|
|  Maria| 29| 80000| 20000|     60000| 20000|
|    Jen| 32| 65000|-15000|     80000|-15000|
+-------+---+------+------+----------+------+



In [21]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Define window specification
df = df.withColumn("id", F.monotonically_increasing_id())
window = Window.orderBy("id")

# Generate the lag of the variable
df = df.withColumn("prev_value", F.lag(df.salary).over(window))

# Compute the difference with lag
df = df.withColumn("diff", F.when(F.isnull(df.salary - df.prev_value), 0)
.otherwise(df.salary - df.prev_value)).drop("id")

df.show()

+-------+---+------+------+----------+------+
|   name|age|salary|  data|prev_value|  diff|
+-------+---+------+------+----------+------+
|  James| 34| 55000| 55000|      null|     0|
|Michael| 30| 70000| 15000|     55000| 15000|
| Robert| 37| 60000|-10000|     70000|-10000|
|  Maria| 29| 80000| 20000|     60000| 20000|
|    Jen| 32| 65000|-15000|     80000|-15000|
+-------+---+------+------+----------+------+



# **21. How to get the day of month, week number, day of year and day of week from a date strings?**

In [5]:
# example data
data = [("2023-05-18","01 Jan 2010",), ("2023-12-31", "01 Jan 2010",)]
df = spark.createDataFrame(data, ["date_str_1", "date_str_2"])

df.show()

+----------+-----------+
|date_str_1| date_str_2|
+----------+-----------+
|2023-05-18|01 Jan 2010|
|2023-12-31|01 Jan 2010|
+----------+-----------+



In [9]:
from pyspark.sql.functions import to_date, dayofmonth, weekofyear, dayofyear, dayofweek

# Assuming df is your DataFrame with the date columns
df_updated = df.withColumn("dayofmonth_1", dayofmonth(to_date(df.date_str_1))) \
               .withColumn("weekofyear_1", weekofyear(to_date(df.date_str_1))) \
               .withColumn("dayofyear_1", dayofyear(to_date(df.date_str_1))) \
               .withColumn("dayofweek_1", dayofweek(to_date(df.date_str_1))) \
               .withColumn("dayofmonth_2", dayofmonth(to_date(df.date_str_2, 'dd MMM yyyy'))) \
               .withColumn("weekofyear_2", weekofyear(to_date(df.date_str_2, 'dd MMM yyyy'))) \
               .withColumn("dayofyear_2", dayofyear(to_date(df.date_str_2, 'dd MMM yyyy'))) \
               .withColumn("dayofweek_2", dayofweek(to_date(df.date_str_2, 'dd MMM yyyy')))
df_updated.show()


+----------+-----------+------------+------------+-----------+-----------+------------+------------+-----------+-----------+
|date_str_1| date_str_2|dayofmonth_1|weekofyear_1|dayofyear_1|dayofweek_1|dayofmonth_2|weekofyear_2|dayofyear_2|dayofweek_2|
+----------+-----------+------------+------------+-----------+-----------+------------+------------+-----------+-----------+
|2023-05-18|01 Jan 2010|          18|          20|        138|          5|           1|          53|          1|          6|
|2023-12-31|01 Jan 2010|          31|          52|        365|          1|           1|          53|          1|          6|
+----------+-----------+------------+------------+-----------+-----------+------------+------------+-----------+-----------+



# **22. How to convert year-month string to dates corresponding to the 4th day of the month?**

In [10]:
# example dataframe
df = spark.createDataFrame([('Jan 2010',), ('Feb 2011',), ('Mar 2012',)], ['MonthYear'])

df.show()

+---------+
|MonthYear|
+---------+
| Jan 2010|
| Feb 2011|
| Mar 2012|
+---------+



In [13]:
from pyspark.sql.functions import expr, col

# convert YearMonth to date (default to first day of the month)
df = df.withColumn('Date', expr("to_date(MonthYear, 'MMM yyyy')"))

df.show()

df = df.withColumn('Date', expr("date_add(date_sub(Date, day(Date) - 1), 3)"))

df.show()


+---------+----------+
|MonthYear|      Date|
+---------+----------+
| Jan 2010|2010-01-01|
| Feb 2011|2011-02-01|
| Mar 2012|2012-03-01|
+---------+----------+

+---------+----------+
|MonthYear|      Date|
+---------+----------+
| Jan 2010|2010-01-04|
| Feb 2011|2011-02-04|
| Mar 2012|2012-03-04|
+---------+----------+



# **23 How to filter words that contain atleast 2 vowels from a series?**

In [4]:
df = spark.createDataFrame([('Apple',), ('Orange',), ('Plan',) , ('Python',) , ('Money',)], ['Word'])

df.show()

+------+
|  Word|
+------+
| Apple|
|Orange|
|  Plan|
|Python|
| Money|
+------+



In [10]:
from pyspark.sql.functions import col, length, translate

# Filter words that contain at least 2 vowels
df_filtered = df.where((length(col('Word')) - length(translate(col('Word'), 'AEIOUaeiou', ''))) >= 2)
df_filtered.show()

+------+
|  Word|
+------+
| Apple|
|Orange|
| Money|
+------+



# **24. How to filter valid emails from a list?**

In [18]:
# Create a list
data = ['buying books at amazom.com', 'rameses@egypt.com', 'matt@t.co', 'narendra@modi.com']

# Convert the list to DataFrame
df = spark.createDataFrame(data, "string")
df.show(truncate =False)

+--------------------------+
|value                     |
+--------------------------+
|buying books at amazom.com|
|rameses@egypt.com         |
|matt@t.co                 |
|narendra@modi.com         |
+--------------------------+



In [21]:
from pyspark.sql import functions as F

pattern ="^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
df_filtered = df.filter(F.col('value').rlike(pattern))
df_filtered.show()

+-----------------+
|            value|
+-----------------+
|rameses@egypt.com|
|        matt@t.co|
|narendra@modi.com|
+-----------------+



# **25. How to Pivot PySpark DataFrame?**


In [22]:
# Sample data
data = [
(2021, 1, "US", 5000),
(2021, 1, "EU", 4000),
(2021, 2, "US", 5500),
(2021, 2, "EU", 4500),
(2021, 3, "US", 6000),
(2021, 3, "EU", 5000),
(2021, 4, "US", 7000),
(2021, 4, "EU", 6000),
]

# Create DataFrame
columns = ["year", "quarter", "region", "revenue"]
df = spark.createDataFrame(data, columns)
df.show()

+----+-------+------+-------+
|year|quarter|region|revenue|
+----+-------+------+-------+
|2021|      1|    US|   5000|
|2021|      1|    EU|   4000|
|2021|      2|    US|   5500|
|2021|      2|    EU|   4500|
|2021|      3|    US|   6000|
|2021|      3|    EU|   5000|
|2021|      4|    US|   7000|
|2021|      4|    EU|   6000|
+----+-------+------+-------+



In [23]:
from pyspark.sql import functions as F
pivot_df = df.groupBy("year", "quarter").pivot("region").sum("revenue")
pivot_df.show()

+----+-------+----+----+
|year|quarter|  EU|  US|
+----+-------+----+----+
|2021|      2|4500|5500|
|2021|      1|4000|5000|
|2021|      4|6000|7000|
|2021|      3|5000|6000|
+----+-------+----+----+

